# 10.1 GroupBy机制

In [3]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.699354,0.238136
1,a,two,0.571301,1.362663
2,b,one,0.203644,0.559685
3,b,two,-0.680281,-0.104975
4,a,one,0.543426,0.416928


In [4]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.695213
b    0.166669
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.698465
      two    -0.688711
b     one     0.367683
      two    -0.034346
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.698465,-0.688711
b,0.367683,-0.034346


In [10]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -0.688711
            2006    0.367683
Ohio        2005   -0.325192
            2006   -0.780892
Name: data1, dtype: float64

In [11]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.695213,-0.407781
b,0.166669,-1.048556


In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.695213,-0.407781
b,0.166669,-1.048556


In [13]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.698465 -0.372166
     two  -0.688711 -0.479010
b    one   0.367683 -1.575581
     two  -0.034346 -0.521530

In [14]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 10.1.1 遍历各分组

In [15]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.616037 -1.090198
1    a  two -0.688711 -0.479010
4    a  one -0.780892  0.345867
b
  key1 key2     data1     data2
2    b  one  0.367683 -1.575581
3    b  two -0.034346 -0.521530


In [16]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.616037 -1.090198
4    a  one -0.780892  0.345867
('a', 'two')
  key1 key2     data1    data2
1    a  two -0.688711 -0.47901
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.367683 -1.575581
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.034346 -0.52153


In [17]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.367683,-1.575581
3,b,two,-0.034346,-0.521530


In [18]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [19]:
grouped = df.groupby(df.dtypes,axis=1)

In [20]:
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.616037 -1.090198
1 -0.688711 -0.479010
2  0.367683 -1.575581
3 -0.034346 -0.521530
4 -0.780892  0.345867
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 10.1.2 选择一列或所有列的子集

In [21]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.372166
     two  -0.479010
b    one  -1.575581
     two  -0.521530

In [23]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [24]:
s_grouped.mean()

key1  key2
a     one    -0.372166
      two    -0.479010
b     one    -1.575581
      two    -0.521530
Name: data2, dtype: float64

## 10.1.3 使用字典和Series分组

In [25]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a','b','c','d','e'],
                     index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.659479,-0.348815,0.248467,-0.372368,-0.693497
Steve,0.026053,-0.676782,0.106902,-1.370216,1.575612
Wes,-0.813603,NaN,NaN,-1.533463,0.981668
Jim,-0.460662,-0.863587,0.557820,1.133377,-0.968607
Travis,-0.700629,-0.170799,0.739437,-0.092287,0.583203


In [27]:
mapping = {'a':'red','b':'red','c':'blue',
          'd':'blue','e':'red','f':'orange'}
by_column = people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,-0.123901,-1.701792
Steve,-1.263314,0.924883
Wes,-1.533463,0.168065
Jim,1.691196,-2.292857
Travis,0.647150,-0.288225


In [28]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 10.1.4 使用函数分组

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.933744,-1.212402,0.806287,-0.772454,-0.680437
5,0.026053,-0.676782,0.106902,-1.370216,1.575612
6,-0.700629,-0.170799,0.739437,-0.092287,0.583203


In [31]:
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.813603 -0.348815  0.248467 -1.533463 -0.693497
  two -0.460662 -0.863587  0.557820  1.133377 -0.968607
5 one  0.026053 -0.676782  0.106902 -1.370216  1.575612
6 two -0.700629 -0.170799  0.739437 -0.092287  0.583203

## 10.1.5 根据索引层级分组

In [32]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3]],
                                   names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.685544 -0.315437 -0.321870  0.896031 -0.801167
1      0.370629  0.736009  0.042844  1.681252 -1.094239
2     -1.138995 -0.888420  2.001609 -0.907429 -0.076968
3      0.526179 -0.125892  0.198310 -1.924653 -0.127040

In [34]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 数据聚合 

## 10.2.1 逐列及多函数应用

In [3]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [4]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [8]:
grouped_pct.agg(['mean','std',peak_to_peak])

NameError: name 'peak_to_peak' is not defined

In [9]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [10]:
funtions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(funtions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [11]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [14]:
ftuples = [('Durchschnitt','mean'),('Abweichung',np.var)]
grouped['tip_pct','total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [15]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [16]:
grouped.agg({'tip_pct':['min','max','mean','std'],
            'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 10.2.2 返回不含行索引的聚合数据

In [17]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 应用：通用拆分—应用—联合

In [19]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [20]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [23]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [24]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [25]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

## 10.3.1 压缩分组键

In [27]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 10.3.2 分位数与桶分析

In [28]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                     'data2':np.random.randn(1000)})
quartiles = pd.cut(frame.data1,4)
quartiles[:10]

0    (-1.381, 0.0932]
1     (0.0932, 1.567]
2      (1.567, 3.041]
3     (0.0932, 1.567]
4     (0.0932, 1.567]
5    (-1.381, 0.0932]
6    (-1.381, 0.0932]
7    (-1.381, 0.0932]
8     (0.0932, 1.567]
9    (-1.381, 0.0932]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.86, -1.381] < (-1.381, 0.0932] < (0.0932, 1.567] < (1.567, 3.041]]

 def get_states(group):
    return{'min':group.min(),'max':group.max(),
          'count':group.count(),'mean':group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_states).unstack()

In [31]:
grouping = pd.qcut(frame.data1,10,labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_states).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.811638,0.080254,-2.518533
1,100.0,2.074624,-0.047664,-2.451671
2,100.0,2.598379,-0.081696,-2.803394
3,100.0,2.437919,0.131922,-2.236712
4,100.0,2.155801,0.088160,-2.003494
5,100.0,2.624663,0.036058,-1.879653
6,100.0,2.120309,-0.003964,-1.877671
7,100.0,2.365111,0.114868,-2.457369
8,100.0,2.639033,0.074923,-2.159337


## 10.3.3 示例：使用指定分组值填充缺失值

In [32]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.268393
2         NaN
3   -1.358628
4         NaN
5   -0.650866
dtype: float64

In [33]:
s.fillna(s.mean())

0   -0.759296
1   -0.268393
2   -0.759296
3   -1.358628
4   -0.759296
5   -0.650866
dtype: float64

In [34]:
states = ['Ohio','New York','Vermont','Florida',
         'Oregon','Nevada','California','Idaho']
group_key = ['East']*4+['West']*4
data = pd.Series(np.random.randn(8),index=states)
data

Ohio         -0.010952
New York      0.500361
Vermont       0.514300
Florida       0.719550
Oregon        1.280725
Nevada       -0.284265
California   -0.022232
Idaho         0.404202
dtype: float64

In [35]:
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio         -0.010952
New York      0.500361
Vermont            NaN
Florida       0.719550
Oregon        1.280725
Nevada             NaN
California   -0.022232
Idaho              NaN
dtype: float64

In [36]:
data.groupby(group_key).mean()

East    0.402986
West    0.629246
dtype: float64

In [37]:
fill_mean = lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.010952
New York      0.500361
Vermont       0.402986
Florida       0.719550
Oregon        1.280725
Nevada        0.629246
California   -0.022232
Idaho         0.629246
dtype: float64

In [39]:
fill_values = {'East':0.5,'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.010952
New York      0.500361
Vermont       0.500000
Florida       0.719550
Oregon        1.280725
Nevada       -1.000000
California   -0.022232
Idaho        -1.000000
dtype: float64

## 10.3.4 示例：随机采样与排列

In [40]:
suits = ['H','S','C','D']
card_val = (list(range(1,11))+[10]*3)*4
base_names = ['A']+list(range(2,11))+['J','K','Q']
cards = []
for suit in ['H','S','C','D']:
    cards.extend(str(num)+suit for num in base_names)
deck = pd.Series(card_val,index=cards)

In [41]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [42]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

3D    3
8S    8
AC    1
2H    2
7C    7
dtype: int64

In [44]:
get_suit = lambda card:card[-1]
deck.groupby(get_suit).apply(draw,n=2)

C  6C      6
   10C    10
D  3D      3
   6D      6
H  3H      3
   2H      2
S  9S      9
   10S    10
dtype: int64

In [45]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

8C     8
QC    10
3D     3
2D     2
QH    10
9H     9
KS    10
8S     8
dtype: int64

## 10.3.5 示例：分组加权平均和相关性

In [46]:
df = pd.DataFrame({'category':['a','a','a','a',
                               'b','b','b','b'],
                  'data':np.random.randn(8),
                  'weights':np.random.rand(8)})
df

,category,data,weights
0,a,0.315884,0.922715
1,a,1.033880,0.297660
2,a,1.189587,0.926023
3,a,-0.482177,0.300984
4,b,0.310458,0.083757
5,b,0.244713,0.460387
6,b,-1.334198,0.495428
7,b,-0.564146,0.568168


In [47]:
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.635648
b   -0.524253
dtype: float64

# 10.4 数据透视表与交叉表

In [48]:
 tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [50]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [52]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [54]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [55]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',
                aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [56]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],
                columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## 10.4.1 交叉表：crosstab

In [57]:
data

Ohio         -0.010952
New York      0.500361
Vermont            NaN
Florida       0.719550
Oregon        1.280725
Nevada             NaN
California   -0.022232
Idaho              NaN
dtype: float64